# eksplorasi metadata bmkg

_notebook_ ini akan digunakan sebagai acuan pengembangan module `bmkgkit`, berupa pembacaan excel dari bmkg online. 

Berhubung untuk data metadata stasiun sudah disertakan dalam `.bmkgkit`, akan langsung dipakai.

In [1]:
import numpy as np, pandas as pd, pathlib, matplotlib.pyplot as plt, seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2
%aimport hidrokit

In [3]:
from hidrokit.bmkgkit import metadata_station

In [4]:
metadata_station.head()

id_stat      name_stat            type_stat    region  \
0  11010101a       Bakongan  Pos Hujan Kerjasama  Region I   
1  11010201a    Kluet Utara  Pos Hujan Kerjasama  Region I   
2  11010301a  Kluet Selatan  Pos Hujan Kerjasama  Region I   
3  11010401a   Labuhan Haji  Pos Hujan Kerjasama  Region I   
4  11010501a         Meukek  Pos Hujan Kerjasama  Region I   

                   provinsi          kabupaten  lintang_derajat  \
0  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          2.95000   
1  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.59000   
2  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.03000   
3  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.54000   
4  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.54321   

   bujur_derajat  ketinggian_mdpl  soil  exposure  land_use time_zone  
0        97.4700                0   NaN       NaN       NaN    +07:00  
1        97.0000                0   NaN       NaN       NaN       NaN  
2        97.3800                0   NaN       NaN       NaN       NaN  
3        97.0100                0   NaN       NaN       NaN       NaN  
4        97.1523                0   NaN       NaN       NaN       NaN

rasanya tidak perlu digunakan kolom `ketinggian.mdpl soil exposure land.use time.zone`

In [5]:
metadata_station = metadata_station.drop(['ketinggian_mdpl', 'soil', 'exposure', 'land_use', 'time_zone'], axis=1)

In [6]:
metadata_station.head()

id_stat      name_stat            type_stat    region  \
0  11010101a       Bakongan  Pos Hujan Kerjasama  Region I   
1  11010201a    Kluet Utara  Pos Hujan Kerjasama  Region I   
2  11010301a  Kluet Selatan  Pos Hujan Kerjasama  Region I   
3  11010401a   Labuhan Haji  Pos Hujan Kerjasama  Region I   
4  11010501a         Meukek  Pos Hujan Kerjasama  Region I   

                   provinsi          kabupaten  lintang_derajat  bujur_derajat  
0  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          2.95000        97.4700  
1  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.59000        97.0000  
2  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.03000        97.3800  
3  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.54000        97.0100  
4  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.54321        97.1523

Mari kita cek, data yang ada di bmkgonline

In [7]:
print(metadata_station.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8028 entries, 0 to 8027
Data columns (total 8 columns):
id_stat            8028 non-null object
name_stat          8028 non-null object
type_stat          8028 non-null object
region             8028 non-null object
provinsi           8028 non-null object
kabupaten          8028 non-null object
lintang_derajat    8028 non-null float64
bujur_derajat      8028 non-null float64
dtypes: float64(2), object(6)
memory usage: 564.5+ KB
None


In [8]:
print('Jumlah baris yang terdapat dalam metadata: ', metadata_station.shape[0])

Jumlah baris yang terdapat dalam metadata:  8028


In [9]:
print('Jumlah stasiun yang terdata di bmkg (berdasarkan id): ', metadata_station.id_stat.nunique())
print('Jumlah stasiun yang terdata di bmkg (berdasarkan nama): ', metadata_station.name_stat.nunique())

Jumlah stasiun yang terdata di bmkg (berdasarkan id):  8025
Jumlah stasiun yang terdata di bmkg (berdasarkan nama):  7147


Angkanya berbeda, yang seharusnya sama. Dugaan saya adalah, ada stasiun yang tidak memiliki `id`. Atau ada beberapa stasiun yang memiliki `id` serupa (mungkin kah itu?). Tidak mungkin ada nilai `null` / `nan`, karena dari info, diketahui bahwa semua kolom merupakan `non-null`. 

In [10]:
print('Check id stasiun yang sama')
print(metadata_station.id_stat.value_counts(ascending=False).head(5))
id_stasiun_sama = metadata_station.id_stat.value_counts(ascending=False).iloc[:3].index.tolist()

Check id stasiun yang sama
97504        2
51010404a    2
33221801a    2
35101506a    1
35072101a    1
Name: id_stat, dtype: int64


Ternyata ada 3 `id` stasiun yang menyimpan dua informasi stasiun. Stasiun tersebut

In [11]:
stasiun_sama_id = metadata_station.loc[metadata_station['id_stat'].isin(id_stasiun_sama),:]
stasiun_sama_id

id_stat                 name_stat            type_stat      region  \
2418  33221801a           Ungaran / Genuk  Pos Hujan Kerjasama   Region II   
2455  33221801a           Ungaran / Genuk  Pos Hujan Kerjasama   Region II   
4250  51010404a                  Nusasari  Pos Hujan Kerjasama  Region III   
4251  51010404a                   Tetelan  Pos Hujan Kerjasama  Region III   
7231      97504  Stasiun Geofisika Sorong                  UPT    Region V   
7232      97504  Stasiun Geofisika Sorong                  UPT    Region V   

         provinsi      kabupaten  lintang_derajat  bujur_derajat  
2418  Jawa Tengah  Kab. Semarang         -7.14651       110.4057  
2455  Jawa Tengah  Kab. Semarang         -7.14651       110.4057  
4250         Bali  Kab. Jembrana         -8.28000       114.5100  
4251         Bali  Kab. Jembrana         -8.29000       114.5100  
7231  Papua Barat    Kota Sorong         -0.86000       131.2600  
7232  Papua Barat    Kota Sorong         -0.86000       131.2600

In [12]:
def sama_atau_tidak(posisi1):
    posisi2 = posisi1+1
    mask = (~(stasiun_sama_id.iloc[posisi1] == stasiun_sama_id.iloc[posisi2]))
    id_stat = stasiun_sama_id.iloc[posisi1,0]
    total_yang_tidak_sama = mask.sum()
    if total_yang_tidak_sama:
        return (f'stasiun {id_stat} sama dengan stasiun berikutnya? \n\t'
                f'tidak, pada kolom {(mask[mask == True]).index.tolist()}')
    else:
        return (f'stasiun {id_stat} sama dengan stasiun berikutnya? \n\t'
                f'iya')
    
print(sama_atau_tidak(0))
print(sama_atau_tidak(2))
print(sama_atau_tidak(4))

stasiun 33221801a sama dengan stasiun berikutnya? 
	iya
stasiun 51010404a sama dengan stasiun berikutnya? 
	tidak, pada kolom ['name_stat', 'lintang_derajat']
stasiun 97504 sama dengan stasiun berikutnya? 
	iya


Stasiun yang memiliki informasi yang sama akan dihapus dalam tabel

In [13]:
metadata_station = metadata_station.drop_duplicates(subset='id_stat', keep='first').reset_index(drop=True)

In [14]:
metadata_station.head(5)

id_stat      name_stat            type_stat    region  \
0  11010101a       Bakongan  Pos Hujan Kerjasama  Region I   
1  11010201a    Kluet Utara  Pos Hujan Kerjasama  Region I   
2  11010301a  Kluet Selatan  Pos Hujan Kerjasama  Region I   
3  11010401a   Labuhan Haji  Pos Hujan Kerjasama  Region I   
4  11010501a         Meukek  Pos Hujan Kerjasama  Region I   

                   provinsi          kabupaten  lintang_derajat  bujur_derajat  
0  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          2.95000        97.4700  
1  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.59000        97.0000  
2  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.03000        97.3800  
3  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.54000        97.0100  
4  Nanggroe Aceh Darussalam  Kab. Aceh Selatan          3.54321        97.1523

Bagaimana dengan kolom `nama`?

In [15]:
nama_stasiun_sama = metadata_station.name_stat.value_counts()
print(nama_stasiun_sama.head(5),'\n')
print('25 stasiun yang muncul lebih dari satu kali: \n\t',
      ', '.join(nama_stasiun_sama[nama_stasiun_sama != 1].index.tolist()[:25]))
print()
print('Jumlah nama yang memiliki id stasiun lebih dari 1: \n\t',
      (nama_stasiun_sama != 1).sum())

Karanganyar    6
Tanjung        5
Sukadana       5
Sidomulyo      5
Koto Baru      4
Name: name_stat, dtype: int64 

25 stasiun yang muncul lebih dari satu kali: 
	 Karanganyar, Tanjung, Sukadana, Sidomulyo, Koto Baru, Bojong, Sukamaju, Bakung, Beji, Rembang, Guguk, Cubadak, Lambuya, Babadan, Ciawi, Cinangka, Sungai Tarab, Ombilin, Curug, Talawi, Lengkong, Blimbing, Wonorejo, Losari, Makmur

Jumlah nama yang memiliki id stasiun lebih dari 1: 
	 788


Ternyata ada beberapa stasiun yang memiliki nama yang sama, meski memiliki `id` yang berbeda. Karena `id` merupakan identitas utama dari stasiun, maka untuk `nama` yang sama diabaikan. 

Data akan disimpan dalam `.csv` dengan nama `clean_metadata_station`

In [16]:
metadata_station.to_csv('out/clean_metadata_station.csv', index=False)

In [17]:
metadata_station.groupby('type_stat').count()

id_stat  name_stat  region  provinsi  kabupaten  \
type_stat                                                              
Pos Hujan Kerjasama     6993       6993    6993      6993       6993   
UPT                      179        179     179       179        179   
aws                      853        853     853       853        853   

                     lintang_derajat  bujur_derajat  
type_stat                                            
Pos Hujan Kerjasama             6993           6993  
UPT                              179            179  
aws                              853            853

Dari perintah diatas dapat diketahui jumlah tipe stasiun di Indonesia berdasarkan data BMKG:
- 6993 stasiun Pos Hujan Kerjasama
- 179 stasiun UPT
- 853 stasiun aws

## Bedah Jawa Barat
---
Pada pembedahan berikutnya akan lebih fokus dengan lokasi Jawa Barat.

In [18]:
stasiun_jabar = metadata_station[metadata_station.provinsi == 'Jawa Barat']
print('Banyaknya stasiun di Jawa Barat:', stasiun_jabar.shape[0])
stasiun_jabar.head()

Banyaknya stasiun di Jawa Barat: 884


id_stat                   name_stat type_stat     region    provinsi  \
800      96751  Stasiun Meteorologi Citeko       UPT  Region II  Jawa Barat   
801  32010706g      AWS Mekarsari Cibinong       aws  Region II  Jawa Barat   
802  32010103g          AWS Jagorawi Bogor       aws  Region II  Jawa Barat   
803  32012603g           AWS Gunung Geulis       aws  Region II  Jawa Barat   
804  32012512g          AWS Cibereum Bogor       aws  Region II  Jawa Barat   

      kabupaten  lintang_derajat  bujur_derajat  
800  Kab. Bogor         -6.70000      106.85000  
801  Kab. Bogor         -6.41882      106.98320  
802  Kab. Bogor         -6.27635      106.52164  
803  Kab. Bogor         -6.62449      106.86200  
804  Kab. Bogor         -6.71067      106.95010

Diketahui bahwa ada __884__ stasiun yang berlokasikan di Jawa Barat

In [19]:
stasiun_jabar.kabupaten.value_counts()

Kab. Bogor            92
Kab. Sukabumi         78
Kab. Cianjur          76
Kab. Karawang         59
Kab. Indramayu        57
Kab. Cirebon          55
Kab. Bandung          55
Kab. Ciamis           47
Kab. Tasikmalaya      47
Kab. Majalengka       42
Kab. Subang           40
Kab. Bandung Barat    35
Kab. Garut            33
Kab. Kuningan         28
Kab. Bekasi           26
Kab. Purwakarta       25
Kab. Sumedang         22
Kota Bandung          15
Kota Bogor            15
Kota Tasikmalaya      13
Kota Bekasi           10
Kota Banjar            6
Kota Depok             5
Kota Cirebon           2
Kota Sukabumi          1
Name: kabupaten, dtype: int64

Dari informasi diatas diketahui bahwa Kabupaten Bogor memiliki stasiun terbanyak dibandingkan daerah kabupaten lainnya.

In [20]:
stasiun_jabar[stasiun_jabar.kabupaten == 'Kota Bandung']

id_stat                  name_stat            type_stat     region  \
1659      96783  Stasiun Geofisika Bandung                  UPT  Region II   
1660  32730701g        AWS Stageof Bandung                  aws  Region II   
1661  32730501a               Lanud Husein  Pos Hujan Kerjasama  Region II   
1662  32730601a                    Cicendo  Pos Hujan Kerjasama  Region II   
1663  32730801a                    Bengkok  Pos Hujan Kerjasama  Region II   
1664  32731301a               Tekaga Bodas  Pos Hujan Kerjasama  Region II   
1665  33010102a     Lengkong/ Telaga Bodas  Pos Hujan Kerjasama  Region II   
1666  32731302a                   Lengkong  Pos Hujan Kerjasama  Region II   
1667  32732201a                 Margacinta  Pos Hujan Kerjasama  Region II   
1668  32732501a                     Cibiru  Pos Hujan Kerjasama  Region II   
1669  32732601a                  Cigending  Pos Hujan Kerjasama  Region II   
1670  32732602a               Ujung Berung  Pos Hujan Kerjasama  Region II   
1671  32732603a            Panglipur Galih  Pos Hujan Kerjasama  Region II   
1672  32732604a               Gunung Kasur  Pos Hujan Kerjasama  Region II   
1673  32732605a               Ujung Berung  Pos Hujan Kerjasama  Region II   

        provinsi     kabupaten  lintang_derajat  bujur_derajat  
1659  Jawa Barat  Kota Bandung         -6.88356      107.59733  
1660  Jawa Barat  Kota Bandung         -6.83390      107.59740  
1661  Jawa Barat  Kota Bandung         -6.90197      107.57622  
1662  Jawa Barat  Kota Bandung         -6.91205      107.60221  
1663  Jawa Barat  Kota Bandung         -6.86389      107.62440  
1664  Jawa Barat  Kota Bandung         -6.93240      107.63036  
1665  Jawa Barat  Kota Bandung         -6.92601      107.62354  
1666  Jawa Barat  Kota Bandung         -6.93307      107.63028  
1667  Jawa Barat  Kota Bandung         -6.95302      107.64887  
1668  Jawa Barat  Kota Bandung         -6.92421      107.71267  
1669  Jawa Barat  Kota Bandung         -6.89490      107.70846  
1670  Jawa Barat  Kota Bandung         -6.92137      107.73850  
1671  Jawa Barat  Kota Bandung         -6.92470      107.59530  
1672  Jawa Barat  Kota Bandung         -6.90156      107.71025  
1673  Jawa Barat  Kota Bandung         -6.90612      107.70730

In [21]:
stasiun_jabar[['id_stat', 'lintang_derajat', 'bujur_derajat']]

id_stat  lintang_derajat  bujur_derajat
800       96751         -6.70000      106.85000
801   32010706g         -6.41882      106.98320
802   32010103g         -6.27635      106.52164
803   32012603g         -6.62449      106.86200
804   32012512g         -6.71067      106.95010
805   32012509g         -6.69796      106.93510
806   32010806b         -6.16028      107.14294
807   32010101a         -6.46040      106.85640
808   32010102a         -6.46571      106.86646
809   32010301a         -6.53690      106.89050
810   32010302a         -6.40000      106.68330
811   32010303a         -6.56933      106.86375
812   32010501a         -6.59281      106.90193
813   32010502a         -6.57586      106.69392
814   32010601a         -6.51348      107.08080
815   32010602a         -6.50470      107.08367
816   32010603a         -6.46790      107.06590
817   32010604a         -6.46849      107.05594
818   32010605a         -6.61980      107.15800
819   32010606a         -6.51135      107.04756
820   32010701a         -6.41430      106.96210
821   32010702a         -6.40928      106.97621
822   32010703a         -6.45348      106.63672
823   32010704a         -6.42320      107.02448
824   32010705a         -6.45258      106.95727
825   32010801a         -6.57782      107.06113
826   32010802a         -6.51103      107.12720
827   32010803a         -6.52720      107.18390
828   32010804a         -6.73680      106.81510
829   32010805a         -6.50430      107.13300
...         ...              ...            ...
1680  32750303a         -6.25167      106.98513
1681  32750401a         -6.25531      106.97772
1682  32750402a         -6.27187      106.98230
1683  32750801a         -6.27177      106.92772
1684  32750901a         -6.27499      106.98286
1685  32109999a         -6.75000      108.27000
1693  32760101a         -6.40530      106.75890
1694  32760102a         -6.39917      106.78051
1695  32760301a         -6.40530      106.75890
1696  32760302a         -6.38763      106.76274
1697  32760303a         -6.42812      106.83875
1698  32780101f         -7.33082      108.20310
1699  32780301a         -7.32090      108.22259
1700  32780401a         -7.28605      108.19401
1701  32780402a         -7.28975      108.19622
1702  33011001a         -7.38529      108.18790
1703  32780501a         -7.38251      108.20729
1704  33011101a         -7.41334      108.20806
1705  32780502a         -7.41334      108.20806
1706  32780601a         -7.37305      108.21868
1707  32780602a         -7.34202      108.24953
1708  32780603a         -7.36385      108.26591
1709  32780604a         -7.36071      108.26705
1710  32780605a         -7.34361      108.24280
1711  32790101a         -7.37324      108.55860
1712  32790102a         -7.38794      108.54806
1713  32790103a         -7.41214      108.56145
1714  32790104a         -7.36546      108.53253
1715  32790201a         -7.36059      108.60508
1716  32790402a         -7.36626      108.62313

[884 rows x 3 columns]